# Analyse 2: Keyword in Context (KWIC) 

## 0. Importe und Daten-Upload 

In [ ]:
import pandas as pd

In [ ]:
import re

In [ ]:
from pathlib import Path

In [ ]:
conllfiles = Path(r"../data/csv")

In [ ]:
corpus_annotations = {}
for file in conllfiles.iterdir():
    if file.suffix == '.csv':
        #path = os.path.join(conllfiles, filename)  
        data = pd.read_csv(file) 
        corpus_annotations[file.name] = data

In [ ]:
corpus_metadata = pd.read_csv(Path('../data/metadata/QUADRIGA_FS-Text-01_Data01_Corpus-Table.csv'), sep=';')

In [ ]:
corpus_metadata = corpus_metadata.set_index('DC.identifier')

## 1. KWIC-Suche

In [ ]:
class ContextViewer:
    
    def __init__(self, corpus_annotated, corpus_metadata):
        self.prepare_index_dataframe_for_search(corpus_annotated, corpus_metadata)
    
    def prepare_index_dataframe_for_search(self, corpus_annotated, corpus_metadata):
        for filename, annotated_text in corpus_annotated.items():
            txtname = filename.replace('.csv', '')
            if txtname in corpus_metadata.index:
                year, month, day = self.get_date_fname(txtname, corpus_metadata)
                annotated_text['month'] = month
                annotated_text['filename'] = filename
        self.full_df = pd.concat(corpus_annotated.values())
        self.full_df = self.full_df.reset_index()
        print(f'Searching in a corpus of {self.full_df.shape[0]} word occurences')
        
    def get_date_fname(self, txtname, corpus_metadata):  
        date = corpus_metadata.loc[txtname, 'DC.date']
        date = str(date)
        year = date[:4]
        month = date[:7]
        day = date
        return year, month, day 
    
    def get_context_words(self, n_words):
        search_terms = input('Insert a word to search, split by comma if more than one: ')
        if len(search_terms) == 0:
            search_terms = 'Grippe, Krankheit'
        search_terms = search_terms.split(',')
        search_terms = [x.strip() for x in search_terms]
        indices = self.full_df.query(f'Lemma.isin({search_terms})').index
        #print(indices)
        left_contexts = []
        this_words = []
        right_contexts = []
        months = []
        for indice in indices:
            left = self.full_df.iloc[indice-10:indice-1, ]["Token"]
            leftс = left[~left.str.contains('\n')]
            right = self.full_df.iloc[indice+1:indice+10, ]["Token"]
            rightс = right[~right.str.contains('\n')]
            left_contexts.append(' '.join(leftс))
            right_contexts.append(' '.join(rightс))
            this_words.append(self.full_df.iloc[indice, ]["Token"])
            months.append(self.full_df.iloc[indice, ]["month"])
        newdf = pd.DataFrame()
        newdf['left_context'] = left_contexts
        newdf['word'] = this_words
        newdf['right_context'] = right_contexts
        newdf['month'] = months
        return newdf
        
    ## currently unused functionality:
    def get_context_sents(self, n_sentences):
        search_lemma = input('Insert a word to search: ')
        if len(search_lemma) == 0:
            search_lemma = 'Grippe'
        indices = self.full_df.query(f'Lemma=="{search_lemma}"').index
        #print(indices)
        left_contexts = []
        this_sentences = []
        right_contexts = []
        months = []
        for indice in indices:
            #print(indice)
            current_filename = self.full_df.iloc[indice, ]["filename"]
            current_sentence_id = self.full_df.iloc[indice, ]["Sentence_idx"]
            left_context = self.get_sents(direction=-1, 
                                              current_filename=current_filename, 
                                              current_sentence_id=current_sentence_id, 
                                              n_sentences=n_sentences) 
            left_contexts.append(left_context)
            right_context = self.get_sents(direction=1, 
                                               current_filename=current_filename, 
                                               current_sentence_id=current_sentence_id, 
                                               n_sentences=n_sentences) 
            right_contexts.append(right_context)
            this_sentence = self.get_sents(direction=0, 
                                               current_filename=current_filename,
                                               current_sentence_id=current_sentence_id,
                                               n_sentences=1)
            this_sentences.append(this_sentence)
            #this_words.append(self.full_df.iloc[indice, ]["Token"])
            months.append(self.full_df.iloc[indice, ]["month"])
        newdf = pd.DataFrame()
        newdf['left_sentences'] = left_contexts
        newdf['this_sentence'] = this_sentences
        newdf['right_sentences'] = right_contexts
        newdf['month'] = months
        return newdf #.sort_values(by='month')
    
    def get_sents(self, direction, current_filename, current_sentence_id, n_sentences):
        sentences = []
        for n in range(1,n_sentences+1):
            sentence_id = current_sentence_id + (n * direction)
            this_sentence = self.create_sentence(current_filename, sentence_id)
            sentences.append(this_sentence)
        #print(' '.join(sentences))
        return ' '.join(sentences)
    
    def create_sentence(self, current_filename, sentence_id):
        words = self.full_df.query(f'filename=="{current_filename}" and Sentence_idx=={sentence_id}')['Token']
        sentence = ' '.join(words)
        #print(sentence)
        return sentence
        

In [ ]:
kwic = ContextViewer(corpus_annotations, corpus_metadata)

In [ ]:
kwic.get_context_words(n_words=5)